In [1]:
pip install tensorflow keras numpy matplotlib pandas scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras

2024-03-01 12:58:46.607697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 12:58:46.607746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 12:58:46.609277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 12:58:46.617458: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 12:58:47.595134: W tensorflow/compiler/tf2

In [3]:
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import *
import pandas as pd
from keras.models import load_model
from keras.preprocessing import image

### Defining the train images path and partial test images path in which we have to add the image to convert it into full path.


In [4]:
train_path = 'DataSet/Train'
test_path = 'DataSet\Test\\'

In [5]:
train_batches = ImageDataGenerator().flow_from_directory(train_path,target_size = (224,224),classes=['RG_Frente','RG_Verso', 'CPF_Frente', 'CPF_Verso', 'CNH_Frente', 'CNH_Verso'],batch_size = 7)

Found 150 images belonging to 6 classes.


### Importing keras pretrained model (without top layers) resnet50 which we are going to use as base model.


In [6]:
base_model = keras.applications.resnet50.ResNet50( weights = 'imagenet',include_top = False,input_shape = (224,224,3))

2024-03-01 12:59:01.127680: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18455 MB memory:  -> device: 0, name: NVIDIA RTX A4500, pci bus id: 0000:81:00.0, compute capability: 8.6


In [7]:
for layer in base_model.layers:
    layer.trainable = False

### Making keras sequential model by using base model layer as its lower layer and two fully connected dense layer as its top layer which are trainable.

In [12]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1000,activation = 'relu'))
model.add(Dense(6,activation = 'sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 1000)              100353000 
                                                                 
 dense_3 (Dense)             (None, 6)                 6006      
                                                                 
Total params: 123946718 (472.82 MB)
Trainable params: 100359006 (382.84 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
None


### Compile and train the model on epoch = 5

In [13]:
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit(train_batches,
                              epochs=20,
                              steps_per_epoch = 7,
                             )

Epoch 1/20
7/7 [==============================] - 0s 50ms/step - loss: 0.4838 - accuracy: 0.9796
Epoch 2/20
7/7 [==============================] - 0s 48ms/step - loss: 1.4257 - accuracy: 0.9388
Epoch 3/20
7/7 [==============================] - 0s 55ms/step - loss: 0.0253 - accuracy: 1.0000
Epoch 4/20
7/7 [==============================] - 0s 49ms/step - loss: 8.8850e-06 - accuracy: 1.0000
Epoch 5/20
7/7 [==============================] - 0s 45ms/step - loss: 1.6408e-14 - accuracy: 1.0000
Epoch 6/20
7/7 [==============================] - 0s 54ms/step - loss: 1.2521e-13 - accuracy: 1.0000
Epoch 7/20
7/7 [==============================] - 0s 46ms/step - loss: 8.2286e-13 - accuracy: 1.0000
Epoch 8/20
7/7 [==============================] - 0s 74ms/step - loss: 2.4929e-17 - accuracy: 1.0000
Epoch 9/20
7/7 [==============================] - 0s 54ms/step - loss: 0.1632 - accuracy: 0.9796
Epoch 10/20
7/7 [==============================] - 0s 47ms/step - loss: 1.8217e-16 - accuracy: 1.0000
Epoch

In [28]:
model.save('lunar-v4.keras')
print("lunar keras saved")

lunar keras saved


In [24]:
def predict(img_path):
    test_image = image.load_img(img_path, target_size=(224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image, batch_size=1)
    print(result)
    print(result[0])
    print(result[0][1])
    if(result[0][0]==1):
        return "RG Frente"
    elif(result[0][1]==1):
        return "RG Verso"
    elif(result[0][2]==1):
        return "CPF Frente"
    elif(result[0][3]==1):
        return "CPF Verso"
    elif(result[0][4]==1):
        return "CNH Frente"
    elif(result[0][5]==1):
        return "CNH Verso"

In [18]:
import os

def get_and_predict_jpgs(directory_path):
  for filename in os.listdir(directory_path):
      if filename.endswith(".jpg") or filename.endswith(".jpeg"):
          img_path = os.path.join(directory_path, filename)
          prediction = predict(img_path)
          print(f"Image: {filename} - Prediction: {prediction}")

In [25]:
label = predict('DataSet/Predict/rf1.jpg')
print(label)

1/1 [==============================] - 0s 31ms/step
[[1. 0. 0. 0. 0. 0.]]
[1. 0. 0. 0. 0. 0.]
0.0
RG Frente


In [26]:
label = predict('DataSet/Predict/rv1.jpg')
print(label)

1/1 [==============================] - 0s 34ms/step
[[3.7555170e-32 1.0000000e+00 4.8909164e-37 0.0000000e+00 2.3450348e-27
  0.0000000e+00]]
[3.7555170e-32 1.0000000e+00 4.8909164e-37 0.0000000e+00 2.3450348e-27
 0.0000000e+00]
1.0
RG Verso


In [27]:
directory_path = 'DataSet/Predict'
get_and_predict_jpgs(directory_path) 

1/1 [==============================] - 0s 37ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso1.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 26ms/step
[[0.0000000e+00 0.0000000e+00 2.4562238e-21 1.0000000e+00 0.0000000e+00
  0.0000000e+00]]
[0.0000000e+00 0.0000000e+00 2.4562238e-21 1.0000000e+00 0.0000000e+00
 0.0000000e+00]
0.0
Image: cpf_verso0.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 21ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso5.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 23ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso7.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 24ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso6.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 22ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso4.jpg - Prediction: CPF Verso
1